In [ ]:
from eagle_hill_fund.server.tools.financial.data.tool import FinancialDataCompiler
from eagle_hill_fund.server.tools.financial.strategies.tool import BaseStrategyTool

In [ ]:
import pandas as pd

In [ ]:
ehf_data_tool = FinancialDataCompiler()

In [ ]:
exchanges = ehf_data_tool.get_available_exchanges()

In [ ]:
stock_list = ehf_data_tool.screen_companies(exchange="NYSE")

In [ ]:
stock_list = [stock["symbol"] for stock in stock_list]

In [ ]:
partial_stock_list = pd.Series(stock_list).sample(250, random_state=42).tolist()

In [ ]:
price_data = ehf_data_tool.compile_price_data(
    symbols=partial_stock_list,
    from_date="1995-09-01",
    to_date="2025-09-15",
    interval="1day"
)

In [ ]:
# Import required classes
from eagle_hill_fund.server.tools.financial.strategies.tool import BaseStrategyTool, StrategyConfig, Signal
from eagle_hill_fund.server.tools.data.parallelization.tool import ParallelizationTool
from datetime import datetime
from typing import List
import time

# Create a research-grade strategy with ATR-based stops and trend filtering
class ResearchGradeStrategy(BaseStrategyTool):
    """
    Research-grade strategy implementing "small losses, big wins" framework:
    - ATR-anchored stops/targets (0.5 ATR risk, 2.0 ATR reward)
    - Trend filter: only long in bull markets (price > 200 SMA)
    - Entry: MA crossover (20 > 50) in bull market
    - Exit: MA crossunder or trend filter violation
    - Designed for multi-decade, multi-asset research
    - PARALLELIZED for high-performance computing
    """
    
    def __init__(self, config: StrategyConfig):
        super().__init__(config)
        # Initialize parallelization tool
        self.parallel_tool = ParallelizationTool()
        print(f"🚀 Parallelization enabled: {self.parallel_tool.max_workers} workers")
    
    def _calculate_basic_indicators(self, data: pd.DataFrame) -> pd.DataFrame:
        """Override to add 200 SMA for trend filtering."""
        df = super()._calculate_basic_indicators(data)
        
        # Add 200 SMA for trend filtering
        df['sma_200'] = df['close'].rolling(window=200).mean()
        
        return df
    
    def calculate_indicators(self) -> None:
        """
        Calculate technical indicators for all symbols using parallelization.
        Override to implement custom indicators with parallel processing.
        """
        if self.data is None:
            raise ValueError("Data not set. Call set_data() first.")
        
        print("🔄 Calculating indicators in parallel...")
        start_time = time.time()
        
        # Get unique symbols
        symbols = self.data['symbol'].unique().tolist()
        
        # Create a function to calculate indicators for a single symbol
        def calculate_symbol_indicators(symbol: str) -> tuple:
            symbol_data = self.data[self.data['symbol'] == symbol].copy()
            indicators = self._calculate_basic_indicators(symbol_data)
            return symbol, indicators
        
        # Use parallel processing to calculate indicators
        results = self.parallel_tool.smart_parallel_map(
            calculate_symbol_indicators, 
            symbols
        )
        
        # Store results
        for symbol, indicators in results:
            self.indicators[symbol] = indicators
        
        elapsed_time = time.time() - start_time
        print(f"✅ Indicators calculated for {len(self.indicators)} symbols in {elapsed_time:.2f}s")
        print(f"📊 Processing rate: {len(symbols)/elapsed_time:.1f} symbols/sec")

    def generate_signals(self, date: datetime) -> List[Signal]:
        signals = []

        # Get current data for all symbols
        current_data = self.data[self.data['date'] == date]

        for _, row in current_data.iterrows():
            symbol = row['symbol']
            price = row['close']

            # Get indicators for this symbol
            if symbol in self.indicators:
                indicators = self.indicators[symbol]
                current_indicators = indicators[indicators['date'] == date]

                if not current_indicators.empty:
                    sma_20 = current_indicators['sma_20'].iloc[0]
                    sma_50 = current_indicators['sma_50'].iloc[0]
                    sma_200 = current_indicators['sma_200'].iloc[0]
                    atr = current_indicators['atr'].iloc[0]

                    # --- Trend filter: only long if price > 200SMA (bull market regime) ---
                    in_bull_market = pd.notna(sma_50) and price > sma_50

                    # --- Entry/exit logic: MA crossover with trend filter ---
                    if pd.notna(sma_20) and pd.notna(sma_50) and pd.notna(atr):
                        # Entry: Golden cross in bull market
                        if in_bull_market:
                            # ATR-based stop/target: risk 0.5 ATR, target 2.0 ATR
                            stop_loss = price - 0.5 * atr
                            take_profit = price + 5.0 * atr
                            
                            signals.append(Signal(
                                symbol=symbol,
                                timestamp=date,
                                signal_type='buy',
                                strength=1.0,
                                price=price,
                                metadata={
                                    "stop_loss": stop_loss,
                                    "take_profit": take_profit,
                                    "atr": atr,
                                    "risk_reward_ratio": 10.0,  # 2.0 / 0.5
                                    "trend_filter": "bull" if in_bull_market else "bear",
                                    "entry_reason": "golden_cross_bull_market"
                                }
                            ))
                        
                        # Exit: Death cross or trend filter violation
                        elif sma_20 < sma_50 or not in_bull_market:
                            signals.append(Signal(
                                symbol=symbol,
                                timestamp=date,
                                signal_type='sell',
                                strength=1.0,
                                price=price,
                                metadata={
                                    "exit_reason": "death_cross" if sma_20 < sma_50 else "trend_filter_violation",
                                    "sma_20": sma_20,
                                    "sma_50": sma_50,
                                    "sma_200": sma_200,
                                    "atr": atr,
                                    "trend_filter": "bull" if in_bull_market else "bear"
                                }
                            ))

        return signals

# Configure the strategy for research-grade testing with parallelization
config = StrategyConfig(
    name="ResearchGradeStrategy",
    symbols=partial_stock_list,  # Test with all symbols
    initial_capital=1000000,
    position_size_pct=1/len(partial_stock_list),  # Equal weight per position
    max_positions=len(partial_stock_list),
    commission_per_trade=0.0,  # No commission for research
    start_date=datetime(1995, 9, 1),  # Longer backtest period
    end_date=datetime(2025, 9, 25)
)

# Create strategy instance
strategy = ResearchGradeStrategy(config)

In [ ]:
# Set the data for the strategy
strategy.set_data(price_data)


In [ ]:
# Run the backtest with parallelization
print("Starting research-grade backtest with parallelization...")
print("Strategy: ATR-anchored stops (0.5 ATR risk, 2.0 ATR reward) with trend filtering")
print(f"🚀 Using {strategy.parallel_tool.max_workers} workers for maximum performance")
start_time = time.time()
results = strategy.run_backtest()
elapsed_time = time.time() - start_time
print(f"✅ Backtest completed in {elapsed_time:.2f} seconds!")
print(f"📊 Processing rate: {len(strategy.data)/elapsed_time:.0f} rows/sec")


In [ ]:
# Get performance summary
summary = strategy.get_performance_summary()
print("=== RESEARCH-GRADE STRATEGY PERFORMANCE ===")
print(f"Strategy: {summary['strategy_name']}")
print(f"Framework: Small losses (0.5 ATR), Big wins (2.0 ATR)")
print(f"Trend Filter: Only long in bull markets (price > 200 SMA)")
print("=" * 50)
print(f"Initial Capital: ${summary['initial_capital']:,.2f}")
print(f"Final Value: ${summary['final_value']:,.2f}")
print(f"Total Return: {summary['total_return']:.2%}")
print(f"Annualized Return: {summary['annualized_return']:.2%}")
print(f"Volatility: {summary['volatility']:.2%}")
print(f"Sharpe Ratio: {summary['sharpe_ratio']:.2f}")
print(f"Max Drawdown: {summary['max_drawdown']:.2%}")
print(f"Win Rate: {summary['win_rate']:.2%}")
print(f"Total Trades: {summary['total_trades']}")
print("=" * 50)
print("🎯 Research Framework: ATR-anchored risk management with trend filtering")


In [ ]:
# Plot the performance
try:
    strategy.plot_performance()
except Exception as e:
    print(f"Plotting error: {e}")
    print("You can install matplotlib with: pip install matplotlib")

# Show detailed trade analysis
print("\n=== TRADE ANALYSIS ===")
trade_analysis = strategy.get_trade_analysis()
if 'error' not in trade_analysis:
    print(f"Total Trades: {trade_analysis['total_trades']}")
    print(f"Buy Trades: {trade_analysis['buy_trades']}")
    print(f"Sell Trades: {trade_analysis['sell_trades']}")
    print(f"Total Volume: ${trade_analysis['total_volume']:,.2f}")
    print(f"Total Commission: ${trade_analysis['total_commission']:,.2f}")
    print(f"Average Trade Size: ${trade_analysis['avg_trade_size']:,.2f}")
    print(f"Unique Symbols: {trade_analysis['unique_symbols']}")
else:
    print("No trade analysis available")


In [ ]:
# Show signal metadata for research analysis
print("\n=== SIGNAL METADATA ANALYSIS ===")
if strategy.execution_log:
    # Analyze entry reasons
    entry_signals = [log for log in strategy.execution_log if log.get('action') == 'buy']
    exit_signals = [log for log in strategy.execution_log if log.get('action') == 'sell']
    
    print(f"Entry Signals: {len(entry_signals)}")
    print(f"Exit Signals: {len(exit_signals)}")
    
    # Show sample signals with metadata
    if entry_signals:
        print("\nSample Entry Signal:")
        sample_entry = entry_signals[0]
        print(f"  Symbol: {sample_entry['symbol']}")
        print(f"  Price: ${sample_entry['price']:.2f}")
        print(f"  Quantity: {sample_entry['quantity']:.0f}")
        print(f"  Date: {sample_entry['timestamp']}")
    
    if exit_signals:
        print("\nSample Exit Signal:")
        sample_exit = exit_signals[0]
        print(f"  Symbol: {sample_exit['symbol']}")
        print(f"  Price: ${sample_exit['price']:.2f}")
        print(f"  Quantity: {sample_exit['quantity']:.0f}")
        print(f"  Date: {sample_exit['timestamp']}")
else:
    print("No execution log available")

print("\n🎯 Research Framework Summary:")
print("✅ ATR-anchored risk management (0.5 ATR stop, 2.0 ATR target)")
print("✅ Trend filtering (only long in bull markets)")
print("✅ Multi-decade backtesting capability")
print("✅ Multi-asset support")
print("✅ Detailed signal metadata for analysis")


In [ ]:
# Performance comparison and optimization summary
print("\n=== PARALLELIZATION PERFORMANCE SUMMARY ===")
print(f"🔥 Hardware: M3 Ultra Mac Studio ({strategy.parallel_tool.max_workers} workers)")
print(f"📊 Data: {len(strategy.data):,} rows across {strategy.data['symbol'].nunique()} symbols")
print(f"⏱️  Backtest Duration: {elapsed_time:.2f} seconds")
print(f"🚀 Processing Rate: {len(strategy.data)/elapsed_time:.0f} rows/sec")
print(f"💾 Memory Usage: {strategy.parallel_tool.memory_gb:.1f}GB available")

# Calculate speedup vs sequential
estimated_sequential_time = elapsed_time * strategy.parallel_tool.max_workers
speedup = estimated_sequential_time / elapsed_time
print(f"⚡ Estimated Speedup: {speedup:.1f}x vs sequential processing")
print(f"💰 Time Saved: {estimated_sequential_time - elapsed_time:.1f} seconds")

print("\n🎯 Parallelization Benefits:")
print("✅ Indicator calculations distributed across CPU cores")
print("✅ Optimal worker allocation for M3 Ultra")
print("✅ Memory-efficient processing")
print("✅ Scalable to larger datasets")
print("✅ Research-grade performance for multi-decade backtests")
